In [1]:
# from splinter import Browser
from bs4 import BeautifulSoup as bs
import requests
import time
from flask import Flask, render_template, redirect
# from flask_pymongo import PyMongo
import pandas as pd

Scrape Diabetes Dataset

In [2]:
# URL of page to be scraped
# url = "https://www.stateofobesity.org/diabetes/" 
    # above url doesn't always pull, downloaded data to html
url = 'resources/diabetes.html'

tables = pd.read_html(url)
print(tables)

[    Rank                  State Diabetes Rate 2018  95% Confidence Interval  \
0      1         wWest Virginia              16.2%                      NaN   
1      2               BAlabama              14.5%                      NaN   
2      3           YMississippi              14.3%                      NaN   
3      4              CArkansas              14.1%                      NaN   
4      4             RLouisiana              14.1%                      NaN   
5      6             pTennessee              13.9%                      NaN   
6      7              QKentucky              13.8%                      NaN   
7      8        nSouth Carolina              13.3%                      NaN   
8      9               OIndiana              12.7%                      NaN   
9     10               JGeorgia              12.6%                      NaN   
10    11            fNew Mexico              12.5%                      NaN   
11    11                 qTexas              12.5% 

In [3]:
factDF = tables[0]
factDF["State"] = factDF["State"].str[1:]
factDF["Diabetes Rate 2018"] = factDF["Diabetes Rate 2018"].str[:-1]
factDF = factDF.drop(["Trend 1990 - 2018", "95% Confidence Interval"], axis=1)
factDF = factDF.set_index("Rank")
factDF

,State,Diabetes Rate 2018
Rank,,
1,West Virginia,16.2
2,Alabama,14.5
3,Mississippi,14.3
4,Arkansas,14.1
4,Louisiana,14.1
6,Tennessee,13.9
7,Kentucky,13.8
8,South Carolina,13.3
9,Indiana,12.7


Scrape Population Dataset and combine with our dataframe

In [4]:
stateURL = "https://simple.wikipedia.org/wiki/List_of_U.S._states_by_population"

stateTables = pd.read_html(stateURL)
print(stateTables)

[    Rank in the fifty states, 2018  Rank in states & territories  \
0                              NaN                           NaN   
1                              NaN                           NaN   
2                              NaN                           NaN   
3                              NaN                           NaN   
4                              NaN                           NaN   
5                              NaN                           NaN   
6                              NaN                           NaN   
7                              NaN                           NaN   
8                              NaN                           NaN   
9                              NaN                           NaN   
10                             NaN                           NaN   
11                             NaN                           NaN   
12                             NaN                           NaN   
13                             NaN             

In [5]:
stateDF = stateTables[0]
stateDF.head()

,"Rank in the fifty states, 2018",Rank in states & territories,Name,"Population estimate, July 1, 2019[4]","Census population, April 1, 2010[5]",Percent increase from 2010-2019[note 1],"Total seats in the U.S. House of Representatives, 2013–2023","Estimated population per electoral vote, 2018[note 2]","Estimated population per House seat, 2018","Census population per House seat, 2010","Percent of the total U.S. population, 2018[note 3]"
0,NaN,NaN,California,39865590,37254523,6.98%,NaN,719219,746359,702885,12.01%
1,NaN,NaN,Texas,29206997,25145561,15.86%,NaN,755312,797273,698503,8.80%
2,NaN,NaN,Florida,21299325,18801310,13.3%,NaN,734459,788864,696468,6.44%
3,NaN,NaN,New York,19542209,19378102,0.8%,NaN,673869,723786,717707,5.91%
4,NaN,NaN,Pennsylvania,12807060,12702379,0.8%,NaN,640353,711503,705715,3.87%


In [6]:
# drop extra columns
stateDF = stateDF.loc[:,['Name',"Population estimate, July 1, 2019[4]"]]

# rename columns
stateDF=stateDF.rename(columns = {"Name":"State","Population estimate, July 1, 2019[4]":'Population_2019'})
stateDF

,State,Population_2019
0,California,39865590
1,Texas,29206997
2,Florida,21299325
3,New York,19542209
4,Pennsylvania,12807060
5,Illinois,12741080
6,Ohio,11689442
7,Georgia,10519475
8,North Carolina,10383620
9,Michigan,9995915


In [7]:
# get rid of any territories or "Totals" rows
stateDF = stateDF.iloc[:52]
stateDF

,State,Population_2019
0,California,39865590
1,Texas,29206997
2,Florida,21299325
3,New York,19542209
4,Pennsylvania,12807060
5,Illinois,12741080
6,Ohio,11689442
7,Georgia,10519475
8,North Carolina,10383620
9,Michigan,9995915


In [8]:
# find datatypes and convert if needed
stateDF.dtypes

stateDF.Population_2019 = stateDF.Population_2019.astype('int')

C:\Users\monol\Anaconda3\envs\PythonData\lib\site-packages\pandas\core\generic.py:5096: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [9]:
stateDF.dtypes

State              object
Population_2019     int32
dtype: object

In [10]:
#merge stateDF with factDF
factDF = pd.merge(factDF,stateDF, how = 'left', on = 'State')

In [11]:
factDF

,State,Diabetes Rate 2018,Population_2019
0,West Virginia,16.2,1805832
1,Alabama,14.5,4887871
2,Mississippi,14.3,2986530
3,Arkansas,14.1,3013825
4,Louisiana,14.1,4659978
5,Tennessee,13.9,6770010
6,Kentucky,13.8,4468402
7,South Carolina,13.3,5084127
8,Indiana,12.7,6691878
9,Georgia,12.6,10519475


In [12]:
state_abbr_url = "https://simple.wikipedia.org/wiki/U.S._postal_abbreviations"

tables = pd.read_html(state_abbr_url)
state_abbr_df = tables[0].iloc[2:]
state_abbr_df = state_abbr_df.rename(columns={0: "Abbreviation", 1: "State"})
state_abbr_df

,Abbreviation,State
2,AK,Alaska
3,AL,Alabama
4,AR,Arkansas
5,AZ,Arizona
6,CA,California
7,CO,Colorado
8,CT,Connecticut
9,DE,Delaware
10,FL,Florida
11,GA,Georgia


In [13]:
factDF = pd.merge(factDF,state_abbr_df, how = 'left', on = 'State')

In [14]:
#factDF = factDF.set_index("State")

In [15]:
factDF

,State,Diabetes Rate 2018,Population_2019,Abbreviation
0,West Virginia,16.2,1805832,WV
1,Alabama,14.5,4887871,AL
2,Mississippi,14.3,2986530,MS
3,Arkansas,14.1,3013825,AR
4,Louisiana,14.1,4659978,LA
5,Tennessee,13.9,6770010,TN
6,Kentucky,13.8,4468402,KY
7,South Carolina,13.3,5084127,SC
8,Indiana,12.7,6691878,IN
9,Georgia,12.6,10519475,GA


In [16]:
factDF.to_csv("static/diabetes.csv")

In [17]:
factDF.to_json("test.json")